# Setup JPype

The following sets up JPype to be able to work with Java classes from python. First importing jpype itself..

In [1]:
import jpype
import jpype.imports as jimport

Then starting the JVM with our simulation on the classpath, registering the `ch` domain as a python module, alowing python-like imports...

In [2]:
jimport.registerDomain('ch')
jpype.addClassPath("../../java/target/demo-python-java-api-1.0-SNAPSHOT.jar")
jpype.startJVM(jpype.get_default_jvm_path(), "-Djava.class.path=%s" % jpype.getClassPath())

And finally just importing the classes we will use. We are all set!

In [3]:
from ch.dubernet.demopythonapi.simulation import Simulation
from ch.dubernet.demopythonapi.simulation.events import BenchmarkEventHandler, JumpEventHandler, SingEventHandler, SpeakEventHandler, JumpEvent, SingEvent, SpeakEvent

Now just normal python modules we will use here

In [4]:
import cProfile
import time
import pandas as pd

Finish with some constants to use through the experiments

In [5]:
N_AGENTS = 100
N_TIME_STEPS = 100
N_TRIES = 100

# Utility

Now define a simple stopwatch to time execution without profiler overhead. It is not the most reliable benchmarking method, but the results are so clear this should not matter ;-)

In [9]:
class Stopwatch:
    def __init__(self):
        self._records = []
    
    def start(self):
        self._start_time = time.time()
        
    def end(self, **attributes):
        t = time.time()
        self._records.append({**attributes, "time": t - self._start_time})
    
    def to_frame(self):
        return pd.DataFrame.from_records(self._records)

stopwatch = Stopwatch()

# Run Using Java Classes Only

We will now run a simulation using only classes from our Java project.

In [10]:
simulation = Simulation(N_AGENTS, N_TIME_STEPS)
simulation.getEvents().addEventHandler(BenchmarkEventHandler())

for i in range(N_TRIES):
    stopwatch.start()
    simulation.run()
    stopwatch.end(setting="pure java", n_agents=N_AGENTS, n_time_steps=N_TIME_STEPS)
    
print(stopwatch.to_frame().query("setting == \"pure java\"").time.mean())

pr = cProfile.Profile()
pr.enable()
simulation.run()
pr.disable()
pr.print_stats()

0.001558837890625
         27 function calls in 0.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.002    0.002    0.002    0.002 <ipython-input-10-5ba06c0e9284>:13(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-10-5ba06c0e9284>:14(<module>)
        3    0.000    0.000    0.000    0.000 _jclass.py:114(_javaGetAttr)
        2    0.000    0.000    0.000    0.000 codeop.py:132(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:142(__call__)
        2    0.000    0.000    0.000    0.000 hooks.py:207(pre_run_code_hook)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:116(<lambda>)
        2    0.000    0.000    0.000    0.000 interactiveshell.py:1266(user_global_ns)
        2    0.000    0.000    0.002    0.001 interactiveshell.py:3259(run_code)
        2    0.000    0.000    0.000    0.000 ipstruct.py:125(__getattr__)
        2    0.000    0.000    0.000  

# Run Using The Naive Python Implementation

In [11]:
simulation = Simulation(N_AGENTS, N_TIME_STEPS)

class PythonBenchmarkHandler:
    def notifyStart(self):
        pass
    
    def notifyEnd(self):
        pass
    
    def handleEvent(self, event):
        # Here we see one of the caveat of communication between a statically typed and a
        # dynamically typed language: we need to manually perform polymorphic dispatching
        if isinstance(event, JumpEvent):
            event.getAgentId()
            event.getHeight_m()
            event.getTime()
        elif isinstance(event, SingEvent):
            event.getAgentId()
            event.getSong()
            event.getTime()
        elif isinstance(event, SpeakEvent):
            event.getAgentId()
            event.getMessage()
            event.getTime()
        else:
            raise Exception()

handler = jpype.JProxy((JumpEventHandler, SingEventHandler, SpeakEventHandler), inst=PythonBenchmarkHandler())
simulation.getEvents().addEventHandler(handler)

for i in range(N_TRIES):
    stopwatch.start()
    simulation.run()
    stopwatch.end(setting="pure python", n_agents=N_AGENTS, n_time_steps=N_TIME_STEPS)
    
print(stopwatch.to_frame().query("setting == \"pure python\"").time.mean())
            
pr = cProfile.Profile()
pr.enable()
simulation.run()
pr.disable()
pr.print_stats()

0.44403413295745847
         412907 function calls (402910 primitive calls) in 0.570 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    10000    0.231    0.000    0.384    0.000 <ipython-input-11-99afc04f756d>:10(handleEvent)
        1    0.000    0.000    0.000    0.000 <ipython-input-11-99afc04f756d>:4(notifyStart)
        1    0.134    0.134    0.569    0.569 <ipython-input-11-99afc04f756d>:40(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-11-99afc04f756d>:41(<module>)
        1    0.000    0.000    0.000    0.000 <ipython-input-11-99afc04f756d>:7(notifyEnd)
    23269    0.037    0.000    0.068    0.000 _jclass.py:103(_javaInit)
   100000    0.073    0.000    0.091    0.000 _jclass.py:114(_javaGetAttr)
    23269    0.019    0.000    0.024    0.000 _jclass.py:127(_javaSetAttr)
    23269    0.014    0.000    0.028    0.000 _jclass.py:78(_getClassFor)
    10002    0.005    0.000    0.009    0.000 _j

Wow, that is a **MASSIVE** performance overhead!!! Two orders of magnitude slower. This is just too slow.

Looking at the profiler output, you can see _lots_ of calls to JPype methods. This is where we should be able to act!